In [ ]:
rooms = ["bedroom", "livingroom", "kitchen", "bathroom"]

# class_name_equivalenceにあるかもしれませんが
# graphに実際存在している名前
alternative_temp_obj = {
    "remote_control" : "remotecontrol",
    "washing_machine" : "washingmachine",
    "clothes_pile" : "clothespile",# basket_for_clothes
    "pants" : "clothespants",# clothes_pants
    "shirt" : "clothesshirt",# clothes_shirt
    "electrical_outlet" : "powersocket",
    "kitchen_cabinets" : "kitchencabinet",# cupboard
    "cutlery_fork" : "cutleryfork",# fork
    "cutlery_knife" : "cutleryknife",# knife
    "kitchen_table" : "kitchentable",
    "coffee_table" : "coffeetable",
    "diningtable" : "kitchentable",# table
    "wall_phone" : "wallphone",
    "lamp" : "tablelamp",# floor_lamp
    "dish_bowl" : "dishbowl",# bowl
    "bottle_water" : "bottlewater",# water
    "kitchen_counter" : "kitchencounter",
    "lotionbottle" : "hairproduct",# shampoo
    "washing_sponge" : "washingsponge",# sponge
    "picture" : "wallpictureframe",# picture
    "rag" : "towel",
    "bread" : "breadslice",# food_bread
    "garbage_can" : "garbagecan",
    "papertowel" : "napkin",# paper_towel
}

# pythonのcheck_scriptがid_mappingを探す時、変わったclass_name
change_execute_map = {'lightswitch':'light', 'tv':'television', 'remotecontrol':'controller', 'sofa':'couch'}
id_map_used_obj = ['lightswitch'] # id_mappingを使う必要があるオブジェクト　＊＊＊　将来にもずっと更新する必要　＊＊＊

# Selected diagonal cameras for each room.
### Please ref to 'scenario_generate_graph_and_video.ipynb' if you need all four cameras. ###
fixed_cameras = {
    'kitchen':{1:[64,66], 2:[39,41], 3:[28,30], 4:[31,33], 5:[49,51], 6:[62,64], 7:[41,43]},
    'bathroom':{1:[13,15], 2:[9,11], 3:[80,82], 4:[45,47], 5:[80,82], 6:[12,14], 7:[11,13]},
    'bedroom':{1:[39,41], 2:[60,62], 3:[98,100], 4:[65,67], 5:[65,67], 6:[34,36], 7:[90,92]},
    'livingroom':{1:[86,88], 2:[82,84], 3:[47,49], 4:[91,93], 5:[16,18], 6:[84,86], 7:[68,70]}
}

### Supposed to add new objects in these general place for each room of each scene. This should be used mainly in a program which contains transitions of rooms.
### Change this as needed. ###
possible_add_obj_position={
    '1':{'kitchen':'kitchentable', 'bathroom': 'bathroomcounter', 'bedroom': 'coffeetable', 'livingroom':'coffeetable'},
    '2':{'kitchen':'kitchentable', 'bathroom': 'bathroomcounter', 'bedroom': 'desk', 'livingroom':'coffeetable'},
    '3':{'kitchen':'kitchentable', 'bathroom': 'bathroomcounter', 'bedroom': 'desk', 'livingroom':'sofa'},
    '4':{'kitchen':'kitchentable', 'bathroom': 'bathroomcounter', 'bedroom': 'desk', 'livingroom':'coffeetable'},
    '5':{'kitchen':'kitchentable', 'bathroom': 'bathroomcounter', 'bedroom': 'bed', 'livingroom':'tvstand'},
    '6':{'kitchen':'kitchentable', 'bathroom': 'bathroomcounter', 'bedroom': 'coffeetable', 'livingroom':'desk'},
    '7':{'kitchen':'kitchentable', 'bathroom': 'bathroomcounter', 'bedroom': 'desk', 'livingroom':'desk'}
                          }

In [ ]:
# Generate video for a program. Make sure you have the executable open
import sys
import os
import re
import json
import glob
import copy
import time
import random
import shutil
from pathlib import Path

sys.path.append('../simulation/')
sys.path.append('../dataset_utils/')

import add_preconds
import augmentation_utils
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from utils_demo import *
from unity_simulator.comm_unity import UnityCommunication
from unity_simulator import utils_viz

equiv_class = utils.load_name_equivalence()
prefab_class = utils.load_name_equivalence(file_name='../../resources/PrefabClass.json')
camera_mapping_class = utils.load_name_equivalence(file_name='../../resources/camera_data_mapping.json')

In [ ]:
def get_program_file(file_path):
    program_name = ''
    description = ''
    list_of_steps = []
    with open(file_path, "r") as f:
        lines = f.readlines()
        program_name = lines[0].replace("\n", "")
        description = lines[1].replace("\n", "")
        for line in lines[2:]:
            line = line.replace("\n", "")
            if line.startswith("["):
                list_of_steps.append(line)
            else:
                pass

    return program_name, description, list_of_steps

def switchOff(g, script_lines):
    off_list = ['tv', 'television', 'computer', 'faucet']
    for class_name in off_list:
        nodes = find_nodes(g, class_name=class_name)
        for node in nodes:
            states = [x for x in node['states'] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            node['states'] = states
    
    is_first_switchon = True
    is_first_switchoff = True
    for line in script_lines:
        action, objs, indexs = augmentation_utils.parseStrBlock(line)
        if action == 'SWITCHOFF' and objs[0] in ['tv', 'television', 'computer', 'faucet', 'light', 'lightswitch']:
            if is_first_switchoff:
                node = find_nodes(g, id=int(indexs[0]))[0]
                states = node['states']
                if 'OFF' in states:
                    states.remove('OFF')
                    states.append('ON')
                elif 'ON' not in states:
                    states.append('ON')
                node['states'] = states
            is_first_switchon = False
            if objs[0] in ['tv', 'television', 'computer', 'faucet']:
                comm.expand_scene(graph)
        elif action == 'SWITCHON' and objs[0] in ['tv', 'television', 'computer', 'faucet', 'light', 'lightswitch']:
            if is_first_switchon:
                node = find_nodes(g, id=int(indexs[0]))[0]
                states = node['states']
                if 'ON' in states:
                    states.remove('ON')
                    states.append('OFF')
                elif 'OFF' not in states:
                    states.append('OFF')
                node['states'] = states
            is_first_switchoff = False
            if objs[0] in ['light', 'lightswitch']:
                comm.expand_scene(graph)
    return g

#ランダムでid2000以降のobjectを追加してしまうので、削除する
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

def final_json_update(unity_json_dict, python_json_dict):
    try:
        new_graph = copy.deepcopy(unity_json_dict)
        for new_node in new_graph["nodes"]:
            new_node_id = new_node["id"]
            for graph_state_node in python_json_dict["nodes"]:
                if new_node_id == graph_state_node["id"]:
                    new_node["states"] = []
                    new_node["states"] = graph_state_node["states"]
                    new_node["properties"] = graph_state_node["properties"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [ ]:
def export(script_lines, file_name, activity_name, description, graph_state_list, obj_id_map):
    file_path = "../simulation/Output/scene"+str(scene_id)+"/graph states/"
    if os.path.exists(file_path) == False:
        os.mkdir(file_path)
    file_path = file_path + activity_name + "/"
    if os.path.exists(file_path) == False:
        os.mkdir(file_path)

    script_path = file_path + "activityList-program.txt"
    with open(script_path, 'w') as f:
        for s in script_lines:
            s = "<char0> " + s
            f.write("%s\n" % s)

    description_path = file_path + "program-description.txt"
    with open(description_path, 'w') as f:
        f.write("%s\n" % activity_name)
        f.write("%s\n" % description)

    program_path = file_path + "executed-program.txt"
    with open(program_path, 'w') as f:
        f.write("%s\n" % activity_name)
        f.write("%s\n\n\n" % description)
        for s in script_lines:
            s = "<char0> " + s
            f.write("%s\n" % s)
    
    action_list = []
    duration_list = []
    output_graph_map = {}
    new_graph_state_list = []
    new_graph_state_list.append(copy.deepcopy(graph_state_list[0]))
    for script_line in script_lines:
        action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
        action_list.append(action)

    output_graph_path = "../simulation/Output/scene"+str(scene_id)+"/" + file_name + "/0/Action_????_0_graph.json"
    for graph_path in sorted(glob.glob(output_graph_path)):
        temp_path = graph_path.replace("../simulation/Output/scene"+str(scene_id)+"/"+file_name+"/0\\", "")
        frame_times = int(temp_path.split("_")[1])
        output_graph_map[frame_times] = graph_path

    ftaa_path = "../simulation/Output/scene"+str(scene_id)+"/" + file_name + "/0/ftaa_" + file_name + ".txt"
    with open(ftaa_path, "r") as f:
        ftaa_lines = f.readlines()
        # sometimes FIND don't print out in ftaa file if the FIND object is near avatar
        if 'FIND' in action_list and len(ftaa_lines) < len(action_list):
            action_list.remove('FIND')
        i = 0
        graph_state_index = 1
        for action_index, action in zip(range(len(action_list)), action_list):
            ftaa_index = int(ftaa_lines[i].split(" ")[0])
            ftaa_action = ftaa_lines[i].split(" ")[1]
            if action_index == ftaa_index and action == ftaa_action:
                start_frame = ftaa_lines[i].split(" ")[2]
                end_frame = ftaa_lines[i].split(" ")[3]
                i += 1
            elif i+1 < len(ftaa_lines): # look for the action in the next one line of ftaa file
                start_frame = ftaa_lines[i].split(" ")[2]
                ftaa_index = int(ftaa_lines[i+1].split(" ")[0])
                if action_index == ftaa_index: # ftaa has two lines for an action
                    if i+2 < len(ftaa_lines): # look for the action in the next two lines of ftaa file
                        ftaa_index = int(ftaa_lines[i+2].split(" ")[0])
                        if action_index == ftaa_index: # ftaa has three lines for an action
                            end_frame = ftaa_lines[i+2].split(" ")[3]
                            i += 3
                        else:
                            end_frame = ftaa_lines[i+1].split(" ")[3]
                            i += 2
                    else:
                        end_frame = ftaa_lines[i+1].split(" ")[3]
                        i += 2
                else:
                    end_frame = ftaa_lines[i].split(" ")[3]
                    i += 1
            else:
                start_frame = ftaa_lines[i].split(" ")[2]
                end_frame = ftaa_lines[i].split(" ")[3]
                i += 1
            duration_frame = int(end_frame) - int(start_frame)
            # constant values are derived based on ffmpeg command line, used for video generating. 
            # 2.07 is the factor of division of input by output framerate. 30 is the output framerate.
            duration = (duration_frame * 2.07) / 30
            duration_list.append(str(duration))
            
            global graph_output_interval
            frame_counter = 0
            while (True):
                check_value = int(end_frame) - frame_counter
                if check_value < graph_output_interval:
                    if action_index == (len(action_list)-1) and int(end_frame) == frame_counter:
                        frame_counter -= graph_output_interval
                    with open(output_graph_map[frame_counter], 'r', encoding='utf-8-sig', errors='ignore') as json_file:
                        g = final_json_update(json.load(json_file), graph_state_list[graph_state_index])
                        new_graph_state_list.append(copy.deepcopy(g))
                    graph_state_index += 1
                    json_file.close()
                    break
                else:
                    frame_counter += graph_output_interval

    new_ftaa_path = file_path + "/ftaa_" + file_name + ".txt"
    shutil.copyfile(ftaa_path, new_ftaa_path)

    duration_path = file_path + "duration.txt"
    with open(duration_path, 'w') as f:
        for d in duration_list:
            f.write("%s\n" % d)
    
    file_index = 0
    for graph in new_graph_state_list:
        file_index += 1
        json_path = file_path + "activityList-graph-state-" + '{0:03d}'.format(file_index) + ".json"
        with open(json_path, 'w') as outfile:
            json.dump(graph, outfile, indent=2)
        json_path = ''

In [ ]:
def replace_index(script_line, replace_value1, replace_value2=None):
    start_indexes = [i.start() for i in re.finditer('\(', script_line)]
    end_indexes = [i.start() for i in re.finditer('\)', script_line)]
    if replace_value2 is not None and len(start_indexes) > 1:
        script_line = script_line[:start_indexes[1]+1] + replace_value2 + script_line[end_indexes[1]:]
    script_line = script_line[:start_indexes[0]+1] + replace_value1 + script_line[end_indexes[0]:]
    return script_line

def get_obj_name(current_obj):
    graph_class_names = [node['class_name'] for node in graph['nodes']]
    # find in graph for equaivalence objs
    if equiv_class.get(current_obj) is None:
        if (alternative_temp_obj.get(current_obj) is not None):
            new_obj = alternative_temp_obj[current_obj]
        else:
            new_obj = current_obj # Need to consider if it is this possible?
    else:
        for equ_new_obj in equiv_class[current_obj]:
            if alternative_temp_obj.get(equ_new_obj) is not None:
                equ_new_obj = alternative_temp_obj[equ_new_obj]
            if equ_new_obj in graph_class_names:
                new_obj = equ_new_obj
                break
            else:
                new_obj = equiv_class[current_obj][0]
    return new_obj

# graphにない場合
def replace_obj_and_get_index(script_line, current_obj):
    # class_name_equivalenceにない場合
    if (equiv_class.get(current_obj) is None):
        script_line, obj_index = get_obj_index_from_alternative(script_line, current_obj)
    else:
        new_obj = get_obj_name(current_obj)

        if (alternative_temp_obj.get(new_obj) is None):
            script_line = script_line.replace('<'+current_obj+'>', '<'+new_obj+'>')
            action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
            obj_index = get_obj_index(new_obj, action)
            if (obj_index == '1'):
                add_node_and_relation(new_obj)
                obj_index = get_obj_index(new_obj, action)
        else:
            # equivalenceの値がalternativeにある場合
            script_line, obj_index = get_obj_index_from_alternative(script_line, new_obj)
    return script_line, obj_index

def get_obj_index_from_alternative(script_line, current_obj):
    action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
    if (alternative_temp_obj.get(current_obj) is not None):
        new_obj = alternative_temp_obj[current_obj]
        script_line = script_line.replace('<'+current_obj+'>', '<'+new_obj+'>')
        obj_index = get_obj_index(new_obj, action)
        if (obj_index == '1'):
            # Add unfound object in the entire scene
            add_node_and_relation(new_obj)
            obj_index = get_obj_index(new_obj, action)
    else:
        if (current_obj in (name['className'].lower() for name in prefab_class['prefabClasses'])):
            add_node_and_relation(current_obj)
            obj_index = get_obj_index(current_obj, action)
        else:
            same_objs_diff_room = [node['id'] for node in graph['nodes'] if node['class_name'] == current_obj]
            if len(same_objs_diff_room) >0:
                add_node_and_relation(current_obj)
                obj_index = get_obj_index(current_obj, action)
            else:
                raise Exception(current_obj, 'is not found in VH system / in this entire scene, please make sure name is correct. If the name is correct, please add the object manually.')
    return script_line, obj_index

def get_obj_index(obj, action):
    global current_room
    if obj in rooms:
        current_room = obj
        current_room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == current_room][1 if (scene_id == 3 and current_room == 'bedroom') else 0]
        return str(current_room_id)
    else:
        found_in_program = False
        if action not in ['WALK', 'GRAB', 'FIND']: # obj is in hand / same obj from the previous one
            for k, v in obj_id_map.items():
                if obj in k:
                    found_in_program = True
                    return str(v)
        if found_in_program is False:
            current_room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == current_room][1 if (scene_id == 3 and current_room == 'bedroom') else 0]
            ids_in_room = [edge['from_id'] for edge in graph['edges'] if edge['relation_type'] == 'INSIDE' and edge['to_id'] == current_room_id]
            filtered_node_graph = [node for node in graph['nodes'] if node['id'] in ids_in_room]

            id = [node['id'] for node in filtered_node_graph if node['class_name'] == obj]
            if len(id) > 0:
                return str(id[0])
            else:
                return '1'
# Calculate the maximun ID value and add the new object in graph with maximun ID + 1
# Parameters are :
# obj : name of the object you want to add
# room : room to which you want to add
# place : place to where you want to put in the room
# relation_type : you need to set edge's relation type. Possible values are 'ON' and 'INSIDE'
# place_obj_index : it may usually be 0 unless the PLACE objects are more than one. If they are more than one, you can choose by its index of graph value.
def add_object_out_of_script(obj, room, place, relation_type, place_obj_index):
    global scene_id
    max_id = max(x['id'] for x in graph['nodes']) + 1
    room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == room][1 if (scene_id == 3 and room == 'bedroom') else 0] # change index 0 to 1 when scene 3's bedroom
    ids_in_room = [edge['from_id'] for edge in graph['edges'] if edge['relation_type'] == 'INSIDE' and edge['to_id'] == room_id]
    possible_place_id = [node['id'] for node in graph['nodes'] if node['class_name'] == place]
    place_id = [x for x in possible_place_id if x in ids_in_room][place_obj_index]
    add_node(graph, {'class_name': obj, 
                        'category': '', 
                        'id': max_id, 
                        'properties': [], 
                        'states': []})
    add_edge(graph, max_id, 'INSIDE', room_id)
    add_edge(graph, max_id, relation_type, place_id)
    print(obj, max_id, 'is added to the scene on the ', place, 'inside ', room)
    comm.expand_scene(graph)

# Need to change place name, relation type, place index according to the activity requirements.
# This function is called when there is no activity's object in the current room.
def add_node_and_relation(obj):
    global current_room
    add_object_out_of_script(obj, current_room, possible_add_obj_position[str(scene_id)][current_room], 'ON', 0)

In [ ]:
def correct_script(script_list):
    result_list = []
    global room_name_list
    global current_room
    for script_line in script_list:
        action, objs, indexs = augmentation_utils.parseStrBlock(script_line)

        # アクションのみ場合
        if len(objs) == 0:
            result_list.append(script_line)
            continue
        elif indexs[0].isdigit() and indexs[0] != '1': ## Only for accurate ID indication
            if (objs[0] in change_execute_map):
                change_obj_name = change_execute_map[objs[0]]
                keyword = (change_obj_name, int(indexs[0]))
            else:
                keyword = (objs[0], int(indexs[0]))
            program_obj_id_map[keyword+tuple(str(len(program_obj_id_map)))] = int(indexs[0])
            if obj_id_map.get(keyword) is None:
                obj_id_map[keyword] = int(indexs[0])
            
            graph_class_names = [node['class_name'] for node in graph['nodes']]
            if objs[0] not in graph_class_names:
                new_obj = get_obj_name(objs[0])
                script_line = script_line.replace('<'+objs[0]+'>', '<'+new_obj+'>')
            if len(objs) > 1 and objs[1] not in graph_class_names:
                new_obj = get_obj_name(objs[1])
                script_line = script_line.replace('<'+objs[1]+'>', '<'+new_obj+'>')
            result_list.append(script_line)
            if objs[0] in rooms:
                current_room = objs[0]
            if current_room not in room_name_list:
                room_name_list.append(current_room)
            # When the program included newly added object IDs (user search in graph in advance and use it in the program),
            # the following code will add them automatically.
            max_id = max(x['id'] for x in graph['nodes']) + 1
            if int(indexs[0]) == max_id:
                add_object_out_of_script(objs[0], current_room, possible_add_obj_position[str(scene_id)][current_room], 'ON', 0)
            if len(objs) > 1 and int(indexs[1]) == max_id:
                add_object_out_of_script(objs[1], current_room, possible_add_obj_position[str(scene_id)][current_room], 'ON', 0)
        else:
            first_obj = objs[0]
            if len(objs) > 1:
                second_obj = objs[1]
            else:
                second_obj = None
            # ①graphに探す→②class_name_equivalenceに探す→③alternative_temp_objに探す
            first_obj_index = get_obj_index(first_obj, action)
            if first_obj_index == '1':
                script_line, first_obj_index = replace_obj_and_get_index(script_line, first_obj)
            if second_obj is not None and script_line is not None:
                second_obj_index = get_obj_index(second_obj, action)
                if second_obj_index == '1':
                    script_line, second_obj_index = replace_obj_and_get_index(script_line, second_obj)
            else:
                second_obj_index = None

            if script_line is not None:
                script_line = replace_index(script_line, first_obj_index, second_obj_index)
                action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
                if (objs[0] in change_execute_map):
                    change_obj_name = change_execute_map[objs[0]]
                    keyword = (change_obj_name, int(indexs[0]))
                else:
                    keyword = (objs[0], int(indexs[0]))
                program_obj_id_map[keyword+tuple(str(len(program_obj_id_map)))] = int(indexs[0])
                if obj_id_map.get(keyword) is None:
                    obj_id_map[keyword] = int(indexs[0])
                
                result_list.append(script_line)
                if current_room not in room_name_list:
                    room_name_list.append(current_room)
    return result_list

def available_fixed_camera(diagonal_index=None):
    all_included_cameras = []
    for room in room_name_list:
        if diagonal_index is None:
            for camera_index in fixed_cameras[room][scene_id]:
                all_included_cameras.append(camera_index)
        else:
            all_included_cameras.append(fixed_cameras[room][scene_id][diagonal_index])
                
    return all_included_cameras

def choosing_appropriate_cameras(target_range_camera, obj_id_list_for_each_room, all_included_cameras = []):
    unmatched_ids = []
    not_found = False
    for camera in target_range_camera:
        camera_visible_ids = [k for k,v in camera['visible'].items()]
        if set(obj_id_list_for_each_room).issubset(set(camera_visible_ids)): # searching/checking if program obj ids are included withing one camera view
            not_found = False # found a camera index which view includes obj ids
            if camera['camera_index'] not in all_included_cameras:
                all_included_cameras.append(camera['camera_index'])
            break
        else:
            not_found = True
    if not_found: # when all camera indexes don't include this obj id(s)
        i = len(obj_id_list_for_each_room) - 1
        while i>1:
            for camera in target_range_camera:
                camera_visible_ids = [k for k,v in camera['visible'].items()]
                matched_ids = set(obj_id_list_for_each_room).intersection(set(camera_visible_ids))
                if len(matched_ids) >= i:
                    not_found = False
                    if camera['camera_index'] not in all_included_cameras:
                        all_included_cameras.append(camera['camera_index'])
                    unmatched_ids = list(set(obj_id_list_for_each_room) - matched_ids)
                    break
                else:
                    not_found = True
                    i -= 1
            if not_found is False:
                break
    return all_included_cameras, unmatched_ids

def specify_cameras_for_each_room(corresponding_scene_data_list, obj_id_list_for_each_room, room_name, all_included_cameras = []):
    unmatched_ids = []
    target_range_camera = [camera for camera in corresponding_scene_data_list if camera['room'] == room_name and camera['corner_camera'] == False]

    all_included_cameras, unmatched_ids = choosing_appropriate_cameras(target_range_camera, obj_id_list_for_each_room, all_included_cameras)

    searching_index_count = 0
    while len(unmatched_ids) > searching_index_count:
        all_included_cameras, unmatched_ids = choosing_appropriate_cameras(target_range_camera, unmatched_ids, all_included_cameras)
        searching_index_count = len(unmatched_ids)

    if len(unmatched_ids) > 0:
        i = 0
        for camera in target_range_camera:
            for check_id in unmatched_ids:
                if check_id in camera['visible']:
                    if camera['camera_index'] not in all_included_cameras:
                        all_included_cameras.append(camera['camera_index'])
                        i += 1
                    break
            if len(unmatched_ids) == i:
                break

    if len(all_included_cameras) < 1:
        ## setting one camera for each item
        i = 0
        for camera in target_range_camera:
            for check_id in obj_id_list_for_each_room:
                if check_id in camera['visible']:
                    if camera['camera_index'] not in all_included_cameras:
                        all_included_cameras.append(camera['camera_index'])
                        i += 1
                    break
            if len(obj_id_list_for_each_room) == i:
                break
    return all_included_cameras

def available_cameras():
    all_included_cameras = []
    camera_mapping_scene_list = camera_mapping_class['scene']
    corresponding_scene_data_list = camera_mapping_scene_list[scene_id-1][str(scene_id)]
    
    obj_id_list_for_each_room = []
    room_for_current_objs = initial_room
    previous_room = initial_room
    # program_obj_id_map has to be in correct order same with program
    i = 0
    for k, v in program_obj_id_map.items():
        for room in room_name_list:
            if room in k:
                room_for_current_objs = room
                if len(obj_id_list_for_each_room) > 0: # search for camera indexes for previous room's objs
                    all_included_cameras = specify_cameras_for_each_room(corresponding_scene_data_list, obj_id_list_for_each_room, previous_room, all_included_cameras)
                obj_id_list_for_each_room = [] # clear the previous obj id list for new room
                previous_room = room_for_current_objs
        if str(v) not in obj_id_list_for_each_room: # filter the duplicated obj id for each room
            obj_id_list_for_each_room.append(str(v))
        i += 1
        if len(program_obj_id_map) == i:
            all_included_cameras = specify_cameras_for_each_room(corresponding_scene_data_list, obj_id_list_for_each_room, room_for_current_objs, all_included_cameras)
    
    # Adding two wide camera in possible rooms
    all_included_cameras += available_fixed_camera()
    return all_included_cameras

In [ ]:
scene_id = 6
program_files = []
data_path = "../dataset/Test/scene"+str(scene_id)+"/*.txt"
for file_path in sorted(glob.glob(data_path)):
    file_name = file_path.replace(file_path[0:file_path.rfind("\\")+1], "")
    
    program_name, description, list_of_steps = get_program_file(file_path)
    program_files.append({
        "file_name":file_name,
        "name": program_name,
        "description": description,
        "list_of_steps": list_of_steps
    })

In [ ]:
comm = UnityCommunication(timeout_wait=18000)

for program in program_files:
    print('\nStarting',program["file_name"])
    message = comm.reset(scene_id - 1)
    su, graph = comm.environment_graph()
    program_name = program["name"]
    script_lines = program["list_of_steps"]

    # *** the following is the object adding code to environment and it has to be used here not other places.
    # *** if you need more than one object, you can use several times
    # *** the purpose of this line is to add new object even the same object is existed in same room or any other different room
#     add_object_out_of_script('book', 'livingroom', 'bookshelf', 'INSIDE', 0)

    initial_room = 'livingroom' # Initial Room : Need to change according to activity　＊＊＊＊＊＊＊
    current_room = initial_room
    obj_id_map = {}
    program_obj_id_map = {}
    room_name_list = []
    room_name_list.append(current_room)
    graph_output_interval = 5 # Control your output graph state json frequency
    script_lines = correct_script(script_lines)
    graph = switchOff(graph, script_lines)
    comm.add_character(initial_room=initial_room)
    su, graph = comm.environment_graph()

    try:
        preconds = add_preconds.get_preconds_script(script_lines).printCondsJSON()
    except Exception as ex:
        print(ex)
        continue

    print('Preconds\n',preconds,'\n')

    modified_graph = check_programs.translate_graph_dict_nofile(graph)
    is_id_map_used = False
    if len([k[0] for k,v in obj_id_map.items() if k[0] in id_map_used_obj]) > 0:
        is_id_map_used = True
    info = check_programs.check_script(script_lines, preconds, graph_path=None, inp_graph_dict=modified_graph, id_mapping=obj_id_map if is_id_map_used else {})
    message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
    graph_state_list = delete2000(graph_state_list)
    print(message)

    if message != 'Script is executable':
        print(message)
        continue

    executed_lines =[]
    for line in script_lines:
        line_to_execute = '<char0> '+line
        executed_lines.append(line_to_execute)
        
    sus_msg, original_graph = comm.environment_graph()
    graph_state_list[0] = final_json_update(original_graph, graph_state_list[0])

    s, message = comm.render_script(
        executed_lines,
        recording=True, 
        find_solution=False, 
        frame_rate=30,
        save_pose_data=True,
        vis_check_object=True, 
        vis_check_character=True,
        vis_check_object_all=True,
        out_graph=True,
        per_frame=graph_output_interval,
        camera_mode=["PERSON_FROM_BACK", "AUTO", "SPECIFIED", "DIAGONAL1", "DIAGONAL2"],
        specified_cameras=available_cameras(),
        diagonal_cameras1=available_fixed_camera(0),
        diagonal_cameras2=available_fixed_camera(1),
        file_name_prefix=program["file_name"][:-4],
        output_folder="Output/scene" + str(scene_id))
    print(str(s) + (str(": " + str(message)) if s==False else ''))

    export(script_lines, program["file_name"][:-4], program['name'], program['description'], graph_state_list, obj_id_map)

In [ ]:
import cv2
import numpy as np
# import imagehash

# function to calculate the amount of difference between two images
def mse(img1, img2):
    h, w = img1.shape
    diff = cv2.subtract(img1, img2)
    err = np.sum(diff**2)
    mse = err/(float(h*w))
    return mse, diff

#### complementing missing images
try:
    for program in program_files:
        program_file_name = program["file_name"][:-4]
        bb_files = "../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0/Action_0000_*_2D.json"
        for x in range(len(sorted(glob.glob(bb_files)))):
            img_path = "../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0/??????_????_" + str(x) + "_normal.png"
            i = 0
            for file_path in sorted(glob.glob(img_path)):
                file_name = file_path.replace("../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0\\", "")
                file_num = file_name.split("_")[1]
                if i != int(file_num):
                    print(str(i) + " " + str(int(file_num)))
                    img = PIL.Image.open(file_path)
                    new_file_name = "../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0/" + "Action_%04d_" + str(x) + "_normal.png"
                    img.save(new_file_name % i)
                    print(file_name)
                    print(file_name + " - 1")
                    print("save")
                    i += 1
                i += 1
except Exception as e:
    print(e)

In [ ]:
import subprocess

dir_path = "../simulation/Output/scene"+str(scene_id)+"/"
output_path = dir_path + "movies/"
if os.path.exists(output_path) == False:
    os.mkdir(output_path)
try:
    for activity in program_files:
        program_file_name = activity["file_name"][:-4]
        bb_files = "../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0/Action_0000_*_2D.json"
        for x in range(len(sorted(glob.glob(bb_files)))):
            file_name = activity["file_name"]
            activity_name = activity["name"]
            file_name = file_name[:-4]
            if os.path.exists(dir_path + file_name) == True:
                sh = subprocess.check_output(["ffmpeg", "-framerate", "14.5", "-i", dir_path + file_name + "/0/Action_%04d_" + str(x) + "_normal.png", "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-r", "30", output_path + activity_name + "_" + str(x) + ".mp4"])
                print(sh)

            img_json_path = dir_path + file_name + "/0/"
            seperate_file_path = img_json_path + activity_name + "_" + str(x) + "/"
            if os.path.exists(seperate_file_path) == False:
                os.mkdir(seperate_file_path)
            desired_files = [fn for fn in os.listdir(img_json_path) if "_"+str(x)+"_" in fn and "graph.json" not in fn]
            for each_file in desired_files:
                shutil.move(img_json_path+each_file, seperate_file_path)
except Exception as e:
    print(e)

In [ ]:
# Ignore this block, this is only for reason of searching and printing graph
sceneNo = 6
room_name = ''

# Search only nodes for each room or search nodes and edges for whole scene
def searchInGraph (sceneNo, test_room_name):
    comm = UnityCommunication()
    message = comm.reset(sceneNo-1)
    su, graph = comm.environment_graph()
    if test_room_name == '':
        return graph['nodes']
    else:
        test_room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == test_room_name][1 if (sceneNo == 3 and test_room_name == 'bedroom') else 0]
        test_ids_in_room = [edge['from_id'] for edge in graph['edges'] if edge['relation_type'] == 'INSIDE' and edge['to_id'] == test_room_id]
        filtered_graph = [node for node in graph['nodes'] if node['id'] in test_ids_in_room]
        print(max(x['id'] for x in graph['nodes']) + 1)
        return filtered_graph
# out_file = open("../simulation/Output/scene"+str(sceneNo)+".json", "w")  
# json.dump(graph, out_file, indent = 6)   
# out_file.close()
print(searchInGraph(sceneNo, room_name))